<a href="https://colab.research.google.com/github/shreyab375/GraphDB-with-Langchain/blob/main/Langchain_with_Graph_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai neo4j

In [5]:
#Create free instance at https://neo4j.com/cloud/platform/aura-graph-database/ and create API keys for free.
#Below key is deleted
NEO4J_URI="neo4j+s://118neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="1W6EhDEyFjlfK"

In [6]:
#Create an environment variable
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [10]:
from langchain_community.graphs import Neo4jGraph
graphs = Neo4jGraph(
        url = NEO4J_URI,
        username = NEO4J_USERNAME,
        password = NEO4J_PASSWORD
        )

In [11]:
#Create an API key for free at groq.com to use free Models
groq_api ="gsk_QXRIZXnB8Yjm" #This won't work

In [61]:
!pip install --quiet langchain_groq langchain.core

In [14]:
from langchain_groq import ChatGroq

In [15]:
llm = ChatGroq(groq_api_key = groq_api, model_name ="Gemma2-9b-It")

In [20]:
from langchain_core.documents import Document

In [22]:
#Randon from Wikipedia
text = """Andrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur
      focusing on machine learning and artificial intelligence (AI).[2] Ng was a cofounder and head of Google Brain
      and was the former Chief Scientist at Baidu, building the company's Artificial Intelligence
      Group into a team of several thousand people.
      Ng is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL).
      Ng has also worked in the field of online education, cofounding Coursera and DeepLearning.AI.
      He has spearheaded many efforts to "democratize deep learning" teaching over 8 million students through his online courses.
      Ng is renowned globally in computer science, recognized in Time magazine's 100 Most Influential People in 2012 and
      Fast Company's Most Creative People in 2014. His influence extends to being named in the Time100 AI Most Influential People in 2023.
  """

In [23]:
#Dividing the documents into chunks
documents = [Document(page_content =text)]

In [24]:
documents

[Document(page_content='Andrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur \n      focusing on machine learning and artificial intelligence (AI).[2] Ng was a cofounder and head of Google Brain \n      and was the former Chief Scientist at Baidu, building the company\'s Artificial Intelligence\n      Group into a team of several thousand people.\n      Ng is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL). \n      Ng has also worked in the field of online education, cofounding Coursera and DeepLearning.AI.\n      He has spearheaded many efforts to "democratize deep learning" teaching over 8 million students through his online courses.\n      Ng is renowned globally in computer science, recognized in Time magazine\'s 100 Most Influential People in 2012 and\n      Fast Company\'s Most Creative People in 2014. His influence extends to being named in the Time1

In [26]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 6.3 MB/s eta 0:00:00


In [31]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [32]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [35]:
#converting the document to GraphDB
graph_doc = llm_transformer.convert_to_graph_documents(documents)

In [36]:
graph_doc

[GraphDocument(nodes=[Node(id='Andrew Yan-Tak Ng', type='Person'), Node(id='Google Brain', type='Organization'), Node(id='Baidu', type='Organization'), Node(id='Stanford University', type='Organization'), Node(id='Stanford Ai Lab', type='Organization'), Node(id='Coursera', type='Organization'), Node(id='Deeplearning.Ai', type='Organization')], relationships=[Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Google Brain', type='Organization'), type='COFOUNDER'), Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Baidu', type='Organization'), type='CHIEF_SCIENTIST'), Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Stanford University', type='Organization'), type='ADJUNCT_PROFESSOR'), Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Stanford Ai Lab', type='Organization'), type='DIRECTOR'), Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Nod

In [38]:
graph_doc[0].nodes

[Node(id='Andrew Yan-Tak Ng', type='Person'),
 Node(id='Google Brain', type='Organization'),
 Node(id='Baidu', type='Organization'),
 Node(id='Stanford University', type='Organization'),
 Node(id='Stanford Ai Lab', type='Organization'),
 Node(id='Coursera', type='Organization'),
 Node(id='Deeplearning.Ai', type='Organization')]

In [39]:
graph_doc[0].relationships

[Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Google Brain', type='Organization'), type='COFOUNDER'),
 Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Baidu', type='Organization'), type='CHIEF_SCIENTIST'),
 Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Stanford University', type='Organization'), type='ADJUNCT_PROFESSOR'),
 Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Stanford Ai Lab', type='Organization'), type='DIRECTOR'),
 Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Coursera', type='Organization'), type='COFOUNDER'),
 Relationship(source=Node(id='Andrew Yan-Tak Ng', type='Person'), target=Node(id='Deeplearning.Ai', type='Organization'), type='COFOUNDER')]

In [42]:
#Adding the graph to schema
graphs.add_graph_documents(graph_doc)

In [48]:
graphs.refresh_schema()
print(graphs.schema)

Node properties:
person {born: INTEGER, name: STRING}
movie {Title: STRING, released: INTEGER}
Person {id: STRING}
Organization {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:COFOUNDER]->(:Organization)
(:Person)-[:CHIEF_SCIENTIST]->(:Organization)
(:Person)-[:ADJUNCT_PROFESSOR]->(:Organization)
(:Person)-[:DIRECTOR]->(:Organization)


In [54]:
#Query the database without knowing Cypher Query Language
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm,graph = graphs,verbose = True)

In [ ]:
chain.invoke({"query":"Who is Andrew Yan-Tak Ng"})